## Using a pre-trained VGG16 model will allow for recongition and extraction of snake along with description and maps.
#### Following a tutorial where they are clasifying the monkey by species.

### I think we will have to tune the paramters of the model so it will recognize the snake along with the description

In [1]:
from keras.layers import Dense , Flatten # for the last layers
from keras.models import Model

In [2]:
#VGG16 model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [3]:
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint

In [4]:
# image augmentaion
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

In [5]:
import numpy as np 
from glob import glob 
import matplotlib.pyplot as plt 
import time

In [6]:
# Parameters
IMG=200
IMG_SIZE = [IMG, IMG]

numOfClasses = 10
batchSize = 32
EPOCHS = 30
PATIENCE=5

In [7]:
# Data
# =====

trainMyImagesFolder = "C:\Build\SnakeImages\\train"
testMyImagesFolder = "C:\Build\SnakeImages\\test"

train_datagen = ImageDataGenerator(rescale = 1. / 255, 
                                    rotation_range = 20 ,
                                    width_shift_range = 0.2 ,
                                    height_shift_range = 0.2 ,
                                    shear_range = 0.2 ,
                                    zoom_range = 0.2 ,
                                    horizontal_flip = True)

training_set = train_datagen.flow_from_directory(trainMyImagesFolder,
                                                shuffle=True,
                                                target_size=IMG_SIZE,
                                                batch_size=batchSize,
                                                class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale = 1. / 255)


test_set = test_datagen.flow_from_directory(testMyImagesFolder,
                                                shuffle=False, #### important
                                                target_size=IMG_SIZE,
                                                batch_size=batchSize,
                                                class_mode = 'categorical')


stepsPerEpochs = np.ceil (training_set.samples / batchSize) # round the result up
validationSteps =np.ceil (test_set.samples / batchSize) 

Found 132 images belonging to 3 classes.
Found 20 images belonging to 3 classes.


In [8]:
# The transfer learning - tune the VGG16 model
myVgg = VGG16(input_shape=IMG_SIZE+ [3],
            weights='imagenet',
            include_top=False) # False means , remove the last fully coneccted layers

#print(myVgg.summary())

58889256/58889256 [==============================] - 35s 1us/step


In [9]:
# we freeze the layers we dont need training
for layer in myVgg.layers:
    layer.trainable = False

In [10]:
# add Flatten layer

PlusFlattenLayer = Flatten()(myVgg.output)

In [11]:
# add the last layer
lastPredictionLayer = Dense(numOfClasses, activation='softmax')(PlusFlattenLayer)

In [12]:
# final model 
model = Model(inputs=myVgg.input , outputs=lastPredictionLayer)
print(model.summary())

model.compile(loss='categorical_crossentropy',
            optimizer='Adam',
            metrics=['accuracy'] )

best_model_file = "C:\Build\SnakeImages\MyBestModel.h5"
bestModel = ModelCheckpoint(best_model_file, monitor='val_accuracy', verbose=1, save_best_only=True)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 200, 200, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 200, 200, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 100, 100, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 100, 100, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 100, 100, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 50, 50, 128)       0     

In [13]:
# train the model
history = model.fit( training_set,
                    validation_data = test_set,
                    epochs=EPOCHS,
                    steps_per_epoch=stepsPerEpochs,
                    validation_steps=validationSteps,
                    verbose=1,
                    callbacks=[bestModel])

Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\olivi\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\olivi\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1041, in launch_instance
      app.start()
    File "C:\Users\olivi\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 724, in start
      self.io_loop.start()
    File "C:\Users\olivi\AppData\Local\Programs\Python\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\olivi\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_

In [14]:
# display the results on charts

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

actualEpochs = range(len(acc))

print("Actual Epochs : "+ str(actualEpochs))

plt.plot(actualEpochs, acc , 'r', label='Training accuracy')
plt.plot(actualEpochs, val_acc , 'b', label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')

plt.show()

NameError: name 'history' is not defined